<a href="https://colab.research.google.com/github/malcolmrite-dsi/WSB_Stock_Screener/blob/main/Reddit_Stock_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the Libraries

In [2]:
!pip install praw

     |████████████████████████████████| 163kB 7.6MB/s 
     |████████████████████████████████| 204kB 10.8MB/s 


Reference: https://www.storybench.org/how-to-scrape-reddit-with-python/



*   PRAW:  Python Reddit API Wrapper, allows easy access to Reddit Data
*   Pandas: To handle the data from the scraping in well formatted tables
*   datetime: To convert the raw date format into an understandable format



In [3]:
import praw
import pandas as pd
import datetime as dt